# Working in Python

## DSC implementation in Python
DSC2 can plug and run both R and Python codes. It also supports interaction between the two languages. Here we show a re-implementation of the [Quick Start example](quick_start.html) in Python. Resource to run this example can be found [here](https://github.com/stephenslab/dsc2/tree/master/vignettes/one_sample_location_python).

The DSC script is almost identical to the Quick Start example:
```yaml
simulate:
    exec: rnorm.py, rt.py
    seed: R(1:10)
    params:
        n: 1000
        true_mean: 0, 1
    return: x, true_mean

estimate:
    exec: mean.py, median.py
    params:
        x: $x
    return: mean

mse:
    exec: MSE.py
    params:
        mean_est: $mean
        true_mean: $true_mean
    return: mse

DSC:
    run: simulate *
         estimate *
         mse
    exec_path: PY/scenarios, PY/methods, PY/scores
    output: dsc_result
```
You may notice the only difference are the executables (and search path): the are now written in Python. 
```python
==> ../vignettes/one_sample_location_python/PY/methods/mean.py <==
import numpy
mean = numpy.mean(x)

==> ../vignettes/one_sample_location_python/PY/methods/median.py <==
import numpy
mean = numpy.median(x)

==> ../vignettes/one_sample_location_python/PY/scores/MSE.py <==
mse = (mean_est-true_mean)**2

==> ../vignettes/one_sample_location_python/PY/scenarios/rnorm.py <==
# produces n random numbers from normal with specified mean
import numpy
x=numpy.random.normal(loc=true_mean, size=n)

==> ../vignettes/one_sample_location_python/PY/scenarios/rt.py <==
# produces n random numbers from t with df=2 and  with specified mean
import numpy
x=true_mean+numpy.random.standard_t(df=2,size=n)
```
To run the DSC,

In [1]:
! dsc settings.dsc -j8

INFO: DSC script exported to settings.html
INFO: Constructing DSC from settings.dsc ...
simulate:1+estimate:1+mse:1: 100% [========================] 3 0.2/s in 00:00:16
simulate:1+estimate:2+mse:1: 100% [========================] 3 0.2/s in 00:00:16
simulate:2+estimate:1+mse:1: 100% [========================] 3 0.2/s in 00:00:17
simulate:2+estimate:2+mse:1: 100% [========================] 3 0.2/s in 00:00:16
DSC: 100% [================================================] 4 0.1/s in 00:01:07
INFO: Building output database dsc_result.rds ...
INFO: DSC complete!
INFO: Elapsed time 67.964 seconds.


Notice that time consumed for Python version is much longer than the Quick Start example. This is result of a compromise to R users. We anticipate the majority of DSC2 users program in R, and as a result we adopt RDS file as the standard format to cache output data. In order for Python to communicate with R, we have implemented interface for Python to read and write RDS files. The interface is less efficient than R's native RDS support, thus increasing the execution time.

## Mixing Python and R codes
The example below shows mix-match style Python and R routines in the same DSC script:
```yaml
simulate:
    exec: rnorm.py, rt.R
    seed: R(1:10)
    params:
        n: 1000
        true_mean: 0, 1
    return: x, true_mean

estimate:
    exec: mean.R, median.py
    params:
        x: $x
    return: mean

mse:
    exec: MSE.py
    params:
        mean_est: $mean
        true_mean: $true_mean
    return: mse

DSC:
    run: simulate *
         estimate *
         mse
    exec_path: PY/scenarios, PY/methods, PY/scores,
               R/scenarios, R/methods
    output: dsc_result
```
To execute:

In [2]:
! dsc settings_mix.dsc -j8

INFO: Checking R library psych ...
INFO: DSC script exported to settings_mix.html
INFO: Constructing DSC from settings_mix.dsc ...
simulate:1+estimate:1+mse:1: 100% [========================] 3 0.2/s in 00:00:12
simulate:1+estimate:2+mse:1: 100% [========================] 3 0.2/s in 00:00:17
simulate:2+estimate:1+mse:1: 100% [========================] 3 0.4/s in 00:00:07
simulate:2+estimate:2+mse:1: 100% [========================] 3 0.2/s in 00:00:14
DSC: 100% [================================================] 4 0.1/s in 00:00:51
INFO: Building output database dsc_result.rds ...
INFO: DSC complete!
INFO: Elapsed time 52.832 seconds.
